In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict

class SearchAndGenerate:
    def __init__(self):
        # BGE-M3 모델 로드
        self.embedding = SentenceTransformer('../bge-m3')
        
        # Qwen2.5-1.5B 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained("../Qwen2.5-1.5B-Instruct/")
        self.model = AutoModelForCausalLM.from_pretrained(
            "../Qwen2.5-1.5B-Instruct/",
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
    def encode_documents(self, documents: List[str]) -> np.ndarray:
        """문서들을 임베딩으로 변환"""
        return self.embedding.encode(documents, normalize_embeddings=True)
    
    def search(self, query: str, documents: List[str], top_k: int = 2) -> List[Dict]:
        """쿼리에 가장 관련성이 높은 문서들을 검색"""
        # 쿼리 임베딩 생성
        query_embedding = self.embedding.encode(query, normalize_embeddings=True)
        
        # 문서 임베딩 생성
        doc_embeddings = self.encode_documents(documents)
        
        # 유사도 계산
        similarities = np.dot(doc_embeddings, query_embedding)
        
        # 상위 k개 문서 선택
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        results = []
        for idx in top_indices:
            results.append({
                "document": documents[idx],
                "score": float(similarities[idx])
            })
            
        return results
    
    def generate_answer(self, query: str, context: str) -> str:
        """주어진 컨텍스트를 바탕으로 답변 생성"""
        prompt = f"""다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: {query}

컨텍스트: {context}

답변:"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )
            
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 프롬프트 부분 제거
        # answer = answer[len(prompt):].strip()
        
        return answer
    
    def search_and_generate(self, query: str, documents: List[str], top_k: int = 3) -> str:
        """검색과 답변 생성을 한 번에 수행"""
        # 관련 문서 검색
        search_results = self.search(query, documents, top_k)
        
        # 검색된 문서들을 컨텍스트로 결합
        context = "\n".join([result["document"] for result in search_results])
        
        # 답변 생성
        answer = self.generate_answer(query, context)
        
        return answer

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
system.search(query, documents, top_k=2)

[{'document': '머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.',
  'score': 0.6861977577209473},
 {'document': '인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.',
  'score': 0.5965269804000854}]

In [5]:
system = SearchAndGenerate()

# 샘플 문서
documents = [
    "인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.",
    "머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.", #-> 스코어 하락
    "딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습하는 기술입니다.",
    "인공지능은 ~~~한 것이고, 머신러닝은 ~~~한 것입니다.", #-> 스코어 상승
]

# 샘플 쿼리
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"

# 검색 및 답변 생성
answer = system.search_and_generate(query, documents)
# print(f"질문: {query}")
print(f"{answer}") 

답변: 다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: 인공지능과 머신러닝의 차이점은 무엇인가요?

컨텍스트: 머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.
인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.
딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습하는 기술입니다.

답변: 인공지능과 머신러닝의 가장 큰 차이는 그 두 가지 모두가 데이터에서 학습하고 예측하거나 결정을做出하는 능력을 가지고 있지만, 이를 어떻게 사용할 수 있는 방식이 다릅니다. 인공지능은 특정 작업 또는 문제를 해결하기 위해 직접적으로 사용되는 알고리즘이나 모델을 개발하려는 시도를 하는 것이 목표입니다. 그러나 머신러닝은 이 과정을 자동화하며, 대체로 데이터를 수집하고 학습시킨 후, 이와 관련된 결과를 예측하거나 결정하는 데 사용됩니다. 또한, 인공지능은 모든 분야에서 사용될 수 있으며, 특히 의료, 금융, 교육 등에서는 많은 활용 사례가 있습니다. 그러나 머신러닝은 인공지능의 일부분이며, 더 정확하게는 데이터와 알고리즘에 대한 이해를 바탕으로 일련의 계산을 수행하는 것입니다. 마지막으로, 인공지능은 인간의 감성적 의사결정이나 신념에 기반한 행동을 포함하지만, 머신러닝은 이러한 요소를 제외하고 일관되고 효율적인 의사결정을 내리는 것을 가능하게 합니다. 따라서 인공지능은 머신러닝과는 다르게, 인간의 감성을 배제하고 대체로 데이터에 기반한 판단을 내리는 능력을 가집니다. 

따라서 인공지능과 머신러닝의 가장 큰 차이점은, 인공지능은 인간의 감성과 생각을 반영하면서 일관되게 의사결정을 내리는 능력을 갖추고 있지만, 머신러닝은 이보다 더 자동화된 방법을 통해 데이터를 학습하고 예측하는 능력이 있습니다. 따라서 인공지능은 인간의 지혜와 경험을 최대한 활용하여 다양한 상황에서 효과적인 의사결정을 내리는 역할을 하지만, 머신러닝은

In [ ]:
f"""다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: {query}

컨텍스트: {context}

답변:"""

In [6]:
print(f"{answer}") 

다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: 인공지능과 머신러닝의 차이점은 무엇인가요?

컨텍스트: 머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.
인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.
딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습하는 기술입니다.

답변: 인공지능과 머신러닝의 가장 큰 차이는 그 두 가지 모두가 데이터에서 학습하고 예측하거나 결정을做出하는 능력을 가지고 있지만, 이를 어떻게 사용할 수 있는 방식이 다릅니다. 인공지능은 특정 작업 또는 문제를 해결하기 위해 직접적으로 사용되는 알고리즘이나 모델을 개발하려는 시도를 하는 것이 목표입니다. 그러나 머신러닝은 이 과정을 자동화하며, 대체로 데이터를 수집하고 학습시킨 후, 이와 관련된 결과를 예측하거나 결정하는 데 사용됩니다. 또한, 인공지능은 모든 분야에서 사용될 수 있으며, 특히 의료, 금융, 교육 등에서는 많은 활용 사례가 있습니다. 그러나 머신러닝은 인공지능의 일부분이며, 더 정확하게는 데이터와 알고리즘에 대한 이해를 바탕으로 일련의 계산을 수행하는 것입니다. 마지막으로, 인공지능은 인간의 감성적 의사결정이나 신념에 기반한 행동을 포함하지만, 머신러닝은 이러한 요소를 제외하고 일관되고 효율적인 의사결정을 내리는 것을 가능하게 합니다. 따라서 인공지능은 머신러닝과는 다르게, 인간의 감성을 배제하고 대체로 데이터에 기반한 판단을 내리는 능력을 가집니다. 

따라서 인공지능과 머신러닝의 가장 큰 차이점은, 인공지능은 인간의 감성과 생각을 반영하면서 일관되게 의사결정을 내리는 능력을 갖추고 있지만, 머신러닝은 이보다 더 자동화된 방법을 통해 데이터를 학습하고 예측하는 능력이 있습니다. 따라서 인공지능은 인간의 지혜와 경험을 최대한 활용하여 다양한 상황에서 효과적인 의사결정을 내리는 역할을 하지만, 머신러닝은 데이터